In [8]:
import pandas as pd 
import numpy as np
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import plotly.express as px

data = pd.read_csv("data/heuristic_performance_final_scratch.csv")
data.set_index("instance", inplace=True)
data

,elshafei_constructive_objective,elshafei_constructive_greedy_local_search_objective,constructive_greedy_local_search_objective,5_multistart_adjacent_swap_optimal_neighbour_objective,5_multistart_adjacent_swap_first_improvement_objective,5_dissimilarity_local_search_adjacent_swap_objective,5_multistart_total_swap_optimal_neighbour_objective,5_multistart_total_swap_first_improvement_objective,5_dissimilarity_local_search_total_swap_objective,10_multistart_adjacent_swap_optimal_neighbour_objective,...,10_dissimilarity_local_search_total_swap_objective,25_multistart_adjacent_swap_optimal_neighbour_objective,25_multistart_adjacent_swap_first_improvement_objective,25_dissimilarity_local_search_adjacent_swap_objective,25_multistart_total_swap_optimal_neighbour_objective,25_multistart_total_swap_first_improvement_objective,25_dissimilarity_local_search_total_swap_objective,grasp_local_search,grasp_simulated_annealing,genetic_algorithm
instance,,,,,,,,,,,,,,,,,,,,,
tai256c,0.016235,0.003676,0.004289,0.168613,0.168613,0.168613,0.003465,0.003465,0.003465,0.158550,...,0.002624,0.166798,0.166798,0.166798,0.003272,0.003272,0.003272,0.002989,0.158515,0.126427
nug16b,0.100000,0.027419,0.054839,0.148387,0.148387,0.148387,0.019355,0.019355,0.019355,0.122581,...,0.054839,0.096774,0.096774,0.096774,0.020968,0.020968,0.020968,0.009677,0.298387,0.146774
tai35b,0.076242,0.019450,0.063860,0.324254,0.324254,0.324254,0.093879,0.093879,0.093879,0.519933,...,0.015643,0.443636,0.443636,0.443636,0.016018,0.016018,0.016018,0.005118,0.579194,0.340723
chr22a,0.133528,0.033788,0.061404,0.434048,0.434048,0.434048,0.091618,0.091618,0.091618,0.306368,...,0.051657,0.351852,0.351852,0.351852,0.096491,0.096491,0.096491,0.058155,1.068876,0.427225
esc16h,0.000000,0.000000,0.000000,0.130522,0.130522,0.130522,0.000000,0.000000,0.000000,0.186747,...,0.000000,0.124498,0.124498,0.124498,0.000000,0.000000,0.000000,0.000000,0.174699,0.014056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
rou12,0.123306,0.056927,0.072017,0.165840,0.165840,0.165840,0.051832,0.051832,0.051832,0.075303,...,0.001376,0.070607,0.070607,0.070607,0.000000,0.000000,0.000000,0.011065,0.193208,0.088686
esc16d,0.000000,0.000000,0.000000,0.500000,0.500000,0.500000,0.000000,0.000000,0.000000,0.500000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.500000,0.375000
tai20b,0.359566,0.010278,0.299035,0.783443,0.783443,0.783443,0.016162,0.016162,0.016162,0.674223,...,0.009204,0.179894,0.179894,0.179894,0.000000,0.000000,0.000000,0.003971,0.852005,0.141792


## What makes a particular instance bad across all algos

In [9]:
kmeans = KMeans(n_clusters=2, n_init=30).fit(data)

In [10]:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(data)

In [11]:
pca_df = pd.merge(data,pd.DataFrame(principalComponents).set_index(data.index), left_index=True, right_index=True)
pca_df['label'] = kmeans.labels_.astype(bool)

In [12]:
px.scatter(pca_df, x = 0, y=1, color='label')

In [14]:
pca_df[pca_df[1] > 4]

,elshafei_constructive_objective,elshafei_constructive_greedy_local_search_objective,constructive_greedy_local_search_objective,5_multistart_adjacent_swap_optimal_neighbour_objective,5_multistart_adjacent_swap_first_improvement_objective,5_dissimilarity_local_search_adjacent_swap_objective,5_multistart_total_swap_optimal_neighbour_objective,5_multistart_total_swap_first_improvement_objective,5_dissimilarity_local_search_total_swap_objective,10_multistart_adjacent_swap_optimal_neighbour_objective,...,25_dissimilarity_local_search_adjacent_swap_objective,25_multistart_total_swap_optimal_neighbour_objective,25_multistart_total_swap_first_improvement_objective,25_dissimilarity_local_search_total_swap_objective,grasp_local_search,grasp_simulated_annealing,genetic_algorithm,0,1,label
instance,,,,,,,,,,,,,,,,,,,,,
tai15b,7.59075,0.007535,0.003899,0.024205,0.024205,0.024205,0.0,0.0,0.0,0.01416,...,0.017004,0.001422,0.001422,0.001422,0.0,0.03909,0.012131,-0.748683,6.613528,True


In [7]:
pca_df[pca_df['label']]

,elshafei_constructive_objective,elshafei_constructive_greedy_local_search_objective,constructive_objective,constructive_greedy_local_search_objective,5_multistart_adjacent_swap_optimal_neighbour_objective,5_multistart_adjacent_swap_first_improvement_objective,5_dissimilarity_local_search_adjacent_swap_objective,5_multistart_total_swap_optimal_neighbour_objective,5_multistart_total_swap_first_improvement_objective,5_dissimilarity_local_search_total_swap_objective,...,25_dissimilarity_local_search_adjacent_swap_objective,25_multistart_total_swap_optimal_neighbour_objective,25_multistart_total_swap_first_improvement_objective,25_dissimilarity_local_search_total_swap_objective,grasp_local_search,grasp_simulated_annealing,genetic_algorithm,0,1,label
instance,,,,,,,,,,,,,,,,,,,,,
chr18a,0.751667,0.500991,7.503694,0.514687,2.000360,2.000360,2.000360,0.397549,0.397549,0.397549,...,1.399531,0.315192,0.315192,0.315192,0.189944,4.026671,1.843936,8.794753,0.290883,True
tai15b,7.590750,0.007535,7.346058,0.003899,0.024205,0.024205,0.024205,0.000000,0.000000,0.000000,...,0.017004,0.001422,0.001422,0.001422,0.000000,0.039090,0.012131,4.051213,8.821431,True
chr15a,1.207357,0.774859,6.325990,0.455335,2.279911,2.279911,2.279911,0.486459,0.486459,0.486459,...,1.236055,0.127728,0.127728,0.127728,0.194018,3.472514,1.258286,7.245220,0.613086,True
chr15b,0.825031,0.278348,6.998248,0.497372,1.822528,1.822528,1.822528,0.029787,0.029787,0.029787,...,1.481101,0.189987,0.189987,0.189987,0.081352,3.457071,0.833542,8.062791,0.405413,True
chr15c,1.030303,0.612795,5.164562,0.261785,1.642045,1.642045,1.642045,0.360059,0.360059,0.360059,...,1.160564,0.314184,0.314184,0.314184,0.241162,3.744318,1.050295,6.184281,0.177408,True
chr12a,0.667714,0.195352,3.643425,0.476759,1.235972,1.235972,1.235972,0.112228,0.112228,0.112228,...,0.421064,0.067002,0.067002,0.067002,0.038107,1.612856,0.718174,3.429021,0.382924,True
esc32g,0.000000,0.000000,4.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.666667,0.000000,0.000000,0.000000,0.000000,3.666667,0.000000,3.448372,0.671916,True
chr25a,0.610643,0.335090,4.282929,0.586934,1.677555,1.677555,1.677555,0.405690,0.405690,0.405690,...,1.555848,0.230242,0.230242,0.230242,0.118019,2.863541,1.724974,5.215339,-0.391244,True
esc32e,0.000000,0.000000,15.000000,0.000000,7.000000,7.000000,7.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,0.000000,20.684420,-1.390415,True


## What makes an instance hard for each algo?

## What is the best algorithm for a particular instance?